<a href="https://colab.research.google.com/github/ohki-yu0225/social_media_analysis/blob/main/text_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ソーシャルメディア分析・入門(3)：テキスト分析演習

【内容】
- データの収集
- 形態素解析
- 頻度分析
- ベクトル表現

---
## ライブラリのインポート

テキスト分析やYoutubeデータの収集に必要なライブラリをインポートする。

In [ ]:
!pip install japanize_matplotlib
!pip install janome
!pip install sentence-transformers

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
import googleapiclient.discovery
from googleapiclient.discovery import build
from janome.tokenizer import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
import umap
import re
import os
import math

---
## データの収集

### Youtube APIによるコメントデータの収集

Youtube Data APIを用いて，動画に対するコメントのデータを取得する。Youtubeでは，動画ごとにIDが割り振られており，IDを指定することでAPIを通じてデータを取得する。

【参考】[Youtube Data API](https://developers.google.com/youtube/v3?hl=ja)


【参考】[Youtube Data APIで動画の情報を収集](https://qiita.com/nbayashi/items/bde26cd04f08de21d552)

In [ ]:
def get_comment(api_key, video_id):
    # Disable OAuthlib"s HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    api_service_name = "youtube"
    api_version = "v3"
    DEVELOPER_KEY = api_key
    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    # 動画情報を取得
    video_request = youtube.videos().list(
        part="snippet,statistics",
        id=video_id
    )
    video_response = video_request.execute()

    # 動画情報を表示
    if video_response["items"]:
        video_info = video_response["items"][0]["snippet"]
        video_stats = video_response["items"][0]["statistics"]

        print("=" * 60)
        print("【動画情報】")
        print("=" * 60)
        print(f"タイトル: {video_info["title"]}")
        print(f"公開者: {video_info["channelTitle"]}")
        print(f"公開日: {video_info["publishedAt"]}")
        print(f"再生回数: {video_stats.get("viewCount", "N/A")}")
        print(f"高評価数: {video_stats.get("likeCount", "N/A")}")
        print(f"コメント数: {video_stats.get("commentCount", "N/A")}")
        print("=" * 60)
        print()

    # コメントを取得
    comments = []
    page_token = None

    while True:
        request = youtube.commentThreads().list(
            part="snippet,replies",
            videoId=video_id,
            pageToken=page_token
        )
        response = request.execute()

        for item in response["items"]:
            # For top-level comments
            comment_snippet = item["snippet"]["topLevelComment"]["snippet"]
            comments.append({
                "author": comment_snippet["authorDisplayName"],
                "comment": comment_snippet["textDisplay"],
                "date": comment_snippet["publishedAt"],
                "like_count": comment_snippet["likeCount"]
            })
            # For replies
            if item["snippet"]["totalReplyCount"] > 0:
                for reply_item in item["replies"]["comments"]:
                    reply_snippet = reply_item["snippet"]
                    comments.append({
                        "author": reply_snippet["authorDisplayName"],
                        "comment": reply_snippet["textDisplay"],
                        "date": reply_snippet["publishedAt"],
                        "like_count": reply_snippet["likeCount"]
                    })

        page_token = response.get("nextPageToken")
        if not page_token:
            break

    comments_df = pd.DataFrame(comments).sort_values("date", ascending=True)
    comments_df = comments_df[comments_df["comment"].str.len() > 20].reset_index(drop=True) # 文字数が20文字以下のテキストは除外

    return comments_df

api_key = "" # ここにAPI Keyを入力
video_id = "7t3lEUrCA14"
comments_df = get_comment(api_key, video_id)
comments_df

### 文字列データのクレンジング

収集したデータには不要な文字列が含まれている場合があるため，前処理としてテキストをクレンジングする。

In [ ]:
for i in comments_df["comment"].head(10):
  print(i)

In [ ]:
comments_df["comment"] = comments_df["comment"].str.normalize("NFKC") #Unicode正規化
comments_df["comment"] = comments_df["comment"].str.replace(r"<.*?>|[a-zA-Z0-9!@#$%^&*()_+={}\[\]:;<>,.?~\\/-]", "", regex=True) #HTMLタグ，数字・記号のみの単語を除去
comments_df["comment"] = comments_df["comment"].str.strip() #先頭や末尾のスペースを削除
comments_df

---
## 形態素解析

前処理したテキストデータに対して，形態素解析を行う。Pythonで実装された形態素解析のためのツールである`Janome`を用いる。

In [ ]:
text = comments_df.iloc[1, 1]
print(text)

In [ ]:
t = Tokenizer()
tokens = t.tokenize(text)
for token in tokens:
  print(token)

---
## 特徴語の抽出

全てのテキストデータに対して，形態素解析を行い，名詞のみを抽出する。

In [ ]:
def tokenize_janome(text):
  tokens = []
  for token in t.tokenize(text):
    pos_info = token.part_of_speech.split(",")
    pos = pos_info[0]
    pos_detail1 = pos_info[1] if len(pos_info) > 1 else ""
    if pos not in ["名詞"]:
      continue
    if pos == "名詞" and pos_detail1 in ["代名詞", "数", "接尾", "非自立", "接続詞的"]:
      continue

    tokens.append(token.surface)
  return " ".join(tokens)

comments_df["tokens"] = comments_df["comment"].apply(tokenize_janome)
comments_df

`scikit-learn`には，テキストデータからベクトル表現を獲得するための関数がある。`CountVecorizer`関数を用いて，出現頻度を特徴量とした単語文書行列を作る。

In [ ]:
# 出現頻度の単語文書行列を作成
vectorizer = CountVectorizer(min_df=2, max_df=0.8)  # 最低2回出現、80%以上の文書に出現する単語は除外
word_doc_matrix = vectorizer.fit_transform(comments_df["tokens"])

# DataFrameに変換
count_matrix = pd.DataFrame(
    word_doc_matrix.toarray(),
    columns=vectorizer.get_feature_names_out(),
    index=comments_df.index
)
count_matrix

`TfidfVectorizer`関数を用いて，TF-IDFを特徴量とした単語文章行列を作る。

In [ ]:
# TFIDFの単語文書行列を作成
vectorizer = TfidfVectorizer(min_df=2, max_df=0.8)  # 最低2回出現、80%以上の文書に出現する単語は除外
word_doc_matrix = vectorizer.fit_transform(comments_df["tokens"])

# DataFrameに変換
tfidf_matrix = pd.DataFrame(
    word_doc_matrix.toarray(),
    columns=vectorizer.get_feature_names_out(),
    index=comments_df.index
)
tfidf_matrix

In [ ]:
# あるテキストにおける出現頻度・Tf-IDF上位5単語を抽出
def show_top_words(dtm, doc_idx, top_n=10):
    row = count_matrix.loc[doc_idx]
    top_words = (
        row[row > 0]
        .sort_values(ascending=False)
        .head(top_n)
    )
    print("テキスト", comments_df["comment"][doc_idx])

    print("\n出現頻度が上位の単語")
    for word, value in top_words.items():
        print(f"{word}: {value}")

    row = tfidf_matrix.loc[doc_idx]
    top_words = (
        row[row > 0]
        .sort_values(ascending=False)
        .head(top_n)
    )
    print("\nTF-IDFが上位の単語")
    for word, value in top_words.items():
        print(f"{word}: {value}")

target_index = 0 # ここの数字を変更
show_top_words(count_matrix, doc_idx=target_index, top_n=5)

---
## ベクトル表現

ベクトル表現間の類似度を計算することで，テキスト間の内容の類似性を定量化できる。`scikit-learn`の`cosine_similarity`関数を用いて，コサイン類似度を計算する。

In [ ]:
target_index = 0
text = comments_df.iloc[target_index, 1]
print(text)

In [ ]:
# TF-IDFによるベクトル表現
target_vector = tfidf_matrix.iloc[target_index, :].values
similarity = cosine_similarity(target_vector.reshape(1, -1),  tfidf_matrix)
comments_df["similarity"] = similarity[0]
comments_df.sort_values("similarity", ascending=False).iloc[1:6, :]

近年はTF-IDFなどの古典的な特徴量だけではなく，BERTやGPTなどの深層学習手法をベクトル表現の獲得に用いることが多い。その例として，`SentenceTransformer`を用いて，BERTによるテキストのベクトル表現を計算し，TF-IDFによるベクトル表現と比較する。

In [ ]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
embeddings = model.encode(comments_df["comment"].to_list(), normalize_embeddings=True)
target_vector = embeddings[target_index]
similarity = cosine_similarity(target_vector.reshape(1, -1),  embeddings)
comments_df["similarity"] = similarity[0]
comments_df.sort_values("similarity", ascending=False).iloc[1:6, :]

In [ ]:
# テキストデータのベクトル表現をUMAPで2次元削減した上で可視化
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

reducer = umap.UMAP(n_components=2, random_state=42, n_jobs=1)
tfidf_2d = reducer.fit_transform(tfidf_matrix.values)
axes[0].scatter(
    tfidf_2d[:, 0],
    tfidf_2d[:, 1],
    alpha=0.6,
    s=50
)
axes[0].set_title('TF-IDF')
axes[0].set_xticks([])
axes[0].set_yticks([])

embeddings_2d = reducer.fit_transform(embeddings)
axes[1].scatter(
    embeddings_2d[:, 0],
    embeddings_2d[:, 1],
    alpha=0.6,
    s=50
)
axes[1].set_title('BERT')
axes[1].set_xticks([])
axes[1].set_yticks([])

plt.tight_layout()
plt.show()

---
## テキスト分類

BERTによるベクトル表現を用いて，テキストデータから類似したテキスト集合をクラスターとして分類する。分類には，k-means法（`scikit-learn`の`kMeans`関数）を用いる。

In [ ]:
n_clusters = 2 # ここにクラスター数を入力
kmeans = KMeans(n_clusters=n_clusters, n_init="auto", random_state=42)
labels = kmeans.fit_predict(embeddings)

# DataFrameに付与
comments_df = comments_df.copy()
comments_df["cluster"] = labels
comments_df["embeddings_1"] = embeddings_2d[:, 0]
comments_df["embeddings_2"] = embeddings_2d[:, 1]

In [ ]:
# クラスターごとに色分けして可視化
plt.figure(figsize=(6, 6))
sns.scatterplot(
    x="embeddings_1",
    y="embeddings_2",
    hue="cluster",
    data=comments_df,
    alpha=0.6,
    s=50
)

plt.xticks([])
plt.yticks([])
plt.xlabel("")
plt.ylabel("")

plt.tight_layout()
plt.show()

それぞれのクラスターのテキスト集合から各単語のTF-IDFの平均値を計算し，上位の単語を示す。

In [ ]:
clusters = sorted(comments_df["cluster"].unique())
n_clusters = len(clusters)

n_cols = 3
n_rows = math.ceil(n_clusters / n_cols)

fig, axes = plt.subplots(
    n_rows, n_cols,
    figsize=(4 * n_cols, 3 * n_rows),
    sharex=False,
    sharey=False
)

# axes を常に 1 次元で扱えるようにする
axes = axes.flatten()

for ax, c in zip(axes, clusters):
    idx = comments_df["cluster"] == c

    word_freq = tfidf_matrix.loc[idx].mean(axis=0).sort_values(ascending=False)
    top10_words = word_freq.head(10)

    ax.barh(range(len(top10_words)), top10_words.values)
    ax.set_yticks(range(len(top10_words)))
    ax.set_yticklabels(top10_words.index)
    ax.set_xlabel("平均TF-IDF")
    ax.set_title(f"Cluster {c}")
    ax.invert_yaxis()

# 余った subplot を消す
for ax in axes[n_clusters:]:
    ax.axis("off")

plt.tight_layout()
plt.show()

---
## 演習：任意のYoutubeの動画コメントのテキスト分析

演習1：動画IDを指定し，Youtube動画のコメントを収集する。

In [ ]:
video_id = "" # ここにVideo IDを入力
comments_df = get_comment(api_key, video_id)
comments_df["comment"] = comments_df["comment"].str.normalize("NFKC") #Unicode正規化
comments_df["comment"] = comments_df["comment"].str.replace(r"<.*?>|[a-zA-Z0-9!@#$%^&*()_+={}\[\]:;<>,.?~\\/-]", "", regex=True) #HTMLタグ，数字・記号のみの単語を除去
comments_df["comment"] = comments_df["comment"].str.strip() #先頭や末尾のスペースを削除
comments_df["tokens"] = comments_df["comment"].apply(tokenize_janome)
comments_df

In [ ]:
# TFIDFの単語文書行列を作成
vectorizer = TfidfVectorizer(min_df=2, max_df=0.8)  # 最低2回出現、80%以上の文書に出現する単語は除外
word_doc_matrix = vectorizer.fit_transform(comments_df["tokens"])

# DataFrameに変換
tfidf_matrix = pd.DataFrame(
    word_doc_matrix.toarray(),
    columns=vectorizer.get_feature_names_out(),
    index=comments_df.index
)
tfidf_matrix

In [ ]:
# BERTによるベクトル表現の獲得
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
embeddings = model.encode(comments_df["comment"].to_list(), normalize_embeddings=True)
reducer = umap.UMAP(n_components=2, random_state=42, n_jobs=1)
embeddings_2d = reducer.fit_transform(embeddings)

演習2：テキスト分類を行い，それぞれのクラスターの特徴語を抽出する。クラスター数を変化させて，コメントデータにどのようなトピックが含まれているかを調べる。

In [ ]:
n_clusters =  # ここにクラスター数を入力
kmeans = KMeans(n_clusters=n_clusters, n_init="auto", random_state=42)
labels = kmeans.fit_predict(embeddings)

# DataFrameに付与
comments_df = comments_df.copy()
comments_df["cluster"] = labels
comments_df["embeddings_1"] = embeddings_2d[:, 0]
comments_df["embeddings_2"] = embeddings_2d[:, 1]

plt.figure(figsize=(6, 6))
sns.scatterplot(
    x="embeddings_1",
    y="embeddings_2",
    hue="cluster",
    data=comments_df,
    alpha=0.6,
    s=50
)

plt.title("BERT")
plt.xticks([])
plt.yticks([])

plt.tight_layout()
plt.show()

In [ ]:
clusters = sorted(comments_df["cluster"].unique())
n_clusters = len(clusters)

n_cols = 3
n_rows = math.ceil(n_clusters / n_cols)

fig, axes = plt.subplots(
    n_rows, n_cols,
    figsize=(4 * n_cols, 3 * n_rows),
    sharex=False,
    sharey=False
)

# axes を常に 1 次元で扱えるようにする
axes = axes.flatten()

for ax, c in zip(axes, clusters):
    idx = comments_df["cluster"] == c

    word_freq = tfidf_matrix.loc[idx].mean(axis=0).sort_values(ascending=False)
    top10_words = word_freq.head(10)

    ax.barh(range(len(top10_words)), top10_words.values)
    ax.set_yticks(range(len(top10_words)))
    ax.set_yticklabels(top10_words.index)
    ax.set_xlabel("平均TF-IDF")
    ax.set_title(f"Cluster {c}")
    ax.invert_yaxis()

# 余った subplot を消す
for ax in axes[n_clusters:]:
    ax.axis("off")

plt.tight_layout()
plt.show()